In [5]:
import numpy as np 

In [30]:
class myLogisticRegression:

  def __init__(self,solver='bgd',eta=0.3,thershold=0.5,n_epochs=100,lampda=0,alpha=1,batch_size=20,t0=200, t1=1000):
    self.solver=solver
    self.eta=eta
    self.n_epochs=n_epochs
    self.lampda=lampda
    self.alpha=alpha
    self.batch_size=batch_size
    self.t0=t0
    self.t1=t1
    self.trained=False
    self.thershold=thershold
    self.cost_lst=list()

  def sigmoid(self,z):
    return 1.0/(1 + np.exp(-z))
  def loss(self,y, y_hat):
    loss = -np.mean(y*(np.log(y_hat)) + (1-y)*np.log(1-y_hat))
    return loss
  def learning_schedule(self,t):
    return self.t0 / (t + self.t1)

  def preprocessing(self,X,y):
    X = np.c_[np.ones((X.shape[0], 1)), X] 
    y = y.reshape(len(y), 1)
    return X,y

#optimization algorithms
  def bgd(self,X,y):
    for iteration in range(self.n_epochs):
      gradients = 1/self.m * X.T.dot(self.sigmoid(X.dot(self.theta)) - y)
      regularization = self.lampda/self.m *( (1-self.alpha) * self.theta+self.alpha * np.sign(self.theta))
      self.theta = self.theta - self.eta * (gradients + regularization)
      cost_value=self.loss(y, self.sigmoid(X.dot(self.theta)))
      self.cost_lst.append(cost_value)

  def mbgd(self,X,y):
    for epoch in range(self.n_epochs):
      shuffled_indices = np.random.permutation(self.m)
      X_b_shuffled = X[shuffled_indices]
      y_shuffled = y[shuffled_indices]
      t=0 #for the learning_schedule
      for i in range(0, self.m, self.batch_size):
          t+= 1 
          xi = X_b_shuffled[i:i+self.batch_size]
          yi = y_shuffled[i:i+self.batch_size]
          regularization = self.lampda/self.batch_size * ((1-self.alpha) * self.theta + self.alpha * np.sign(self.theta))
          gradients = 1/self.batch_size * xi.T.dot(self.sigmoid(xi.dot(self.theta)) - yi)
          self.eta = self.learning_schedule(t)
          self.theta = self.theta - self.eta * (gradients+regularization)
      cost_value=self.loss(y, self.sigmoid(X.dot(self.theta)))
      self.cost_lst.append(cost_value)
  def sgd(self,X,y):
    for epoch in range(self.n_epochs):
      for i in range(self.m):        
          random_index = np.random.randint(self.m)
          xi = X[random_index].reshape(1,3)
          yi = y[random_index].reshape(1,1)
          regularization = self.lampda * ((1-self.alpha) * self.theta+self.alpha * np.sign(self.theta))
          gradients =  xi.T.dot(self.sigmoid(xi.dot(self.theta)) - yi) 
          self.eta = self.learning_schedule(epoch * self.m + i)
          self.theta = self.theta - self.eta * (gradients +regularization)
          
      cost_value=self.loss(y, self.sigmoid(X.dot(self.theta)))
      self.cost_lst.append(cost_value)
#training
  def fit(self,X,y):
    self.theta=np.random.randn(len(X[0])+1, 1)
    self.m=X.shape[0]
    self.trained = True
    X,y=self.preprocessing(X,y)
    
    if self.solver=='bgd':
      self.bgd(X,y)
    elif self.solver=='mbgd':
      self.mbgd(X,y)
    elif self.solver=='sgd':
      self.sgd(X,y)
    else:raise Exception("error wrong solver")

#prediction 
  def predict(self,X):
      pred_class = []
      if self.trained == True:
        X,_=self.preprocessing(X,np.array([0,0,0]))
        pred=self.sigmoid(X.dot(self.theta))
        pred_class = [1 if i > self.thershold else 0 for i in pred]
        return np.array(pred_class)
      else:
        raise Exception("This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.")

  def predict_proba(self,X):
      if self.trained == True:
        X,_=self.preprocessing(X,np.array([0,0,0]))
        return self.sigmoid(X.dot(self.theta))
      else:
        raise Exception("This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.")
  def get_cost(self):
    return np.array(self.cost_lst)   

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from fractions import Fraction as frac
import sys

np.random.seed(42)
from sklearn.datasets import make_classification
X, y = make_classification(n_features=2, n_redundant=0, 
                           n_informative=2, random_state=1, 
                           n_clusters_per_class=1)

In [32]:
model = myLogisticRegression(solver='mbgd')
model.fit(X,y)

In [33]:
pre=model.predict(X)
pre

array([1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1])

In [34]:
model.theta

array([[-0.15348587],
       [-4.61754429],
       [ 0.21202298]])

In [36]:
cost=model.get_cost()
cost

array([0.84781203, 0.56114622, 0.40308215, 0.31018392, 0.25101191,
       0.21065124, 0.18147498, 0.1594107 , 0.14220285, 0.12842872,
       0.11714584, 0.10774141, 0.09975382, 0.09292312, 0.0869965 ,
       0.08180231, 0.07721667, 0.07313412, 0.06947825, 0.06618462,
       0.06320477, 0.06049091, 0.05800946, 0.05573147, 0.05363346,
       0.05169551, 0.04989795, 0.04822663, 0.046668  , 0.0452118 ,
       0.04384753, 0.04256766, 0.04136408, 0.04022925, 0.03915763,
       0.03814455, 0.03718517, 0.03627518, 0.03541053, 0.03458796,
       0.03380397, 0.03305691, 0.03234374, 0.03166235, 0.03101044,
       0.03038579, 0.02978711, 0.02921257, 0.0286608 , 0.0281307 ,
       0.02762058, 0.02712941, 0.02665628, 0.02620021, 0.02576039,
       0.02533567, 0.02492531, 0.02452864, 0.02414515, 0.02377403,
       0.02341472, 0.02306663, 0.02272929, 0.02240207, 0.02208459,
       0.0217763 , 0.02147689, 0.02118612, 0.02090351, 0.02062868,
       0.02036137, 0.02010109, 0.01984784, 0.01960119, 0.01936